In [2]:
#import libraries
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [3]:
df = pd.read_csv('./cereal.csv')
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [4]:
entropy_node = 0 #initialize Entropy
values = df.rating.unique() #Unique objects - 'Yes', 'No'
for value in values:
    fraction = df.rating.value_counts()[value]/len(df.rating)
    entropy_node += -fraction*np.log2(fraction)

In [5]:
display(entropy_node)

6.266786540694905

In [6]:
attribute = 'name'
target_variables = df.rating.unique() #This gives all 'Yes' and 'No'
variables = df[attribute].unique() #This give diffrent features
                                    #in that attribute (Like 'Sweet')
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df[attribute][df[attribute]==variable][df.rating ==target_variable]) #numerator
        den = len(df[attribute][df[attribute]==variable]) #denominator
        fraction = num/(den+eps) #pi
        entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature Loke 'Sweet'
    fraction2 = den/len(df)
    entropy_attribute += -fraction*entropy_each_feature #Sums up all the entropy ETaste

In [9]:
display(abs(entropy_attribute))

0.0

In [10]:
def find_entropy(df):
    Class = df.keys()[-1] #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

def find_entropy_attribue(df,attribute):
    Class = df.keys()[-1]                 #To make the code generic, changing target variable class name
    target_variables = df[Class].unique() #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()   #This gives different features
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
        return abs(entropy2)

def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        #Entropy_att.append(find_entropy_attrobite(df,key))
        IG.append(find_entropy(df)-find_entropy_attribue(df,key))
    return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df, node,value):
    return df[df[node] ==value].reset_index(drop=True)

def buildTree(df,tree=None):
    Class = df.keys()[-1]    #To make the code generic, changing target
                             #class name
    
    #Here we build our decision tree
    
    #Get attribute with maximum information gain
    node = find_winner(df)
    
    #Get distinct value of that attribute e.g Slary is node and Low, Med
    #and High are values
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree
    if tree is None:
        tree={}
        tree[node] = {}
        
    #We make Loop to construct a tree by calling this function recursively.
    #in this we check if the subset is pure and stops if it is pure.
    
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['rating'],return_counts=True)
        
        if len(counts)==1: #checking purity of subset
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) #calling the function recursively
    return tree       

In [11]:
tree = buildTree(df)

In [12]:
import pprint
pprint.pprint(tree)

{'name': {'100% Bran': 68.402973,
          '100% Natural Bran': 33.983679,
          'All-Bran': 59.425505,
          'All-Bran with Extra Fiber': 93.704912,
          'Almond Delight': 34.384843,
          'Apple Cinnamon Cheerios': 29.509541,
          'Apple Jacks': 33.174094,
          'Basic 4': 37.038562,
          'Bran Chex': 49.120253,
          'Bran Flakes': 53.313813,
          "Cap'n'Crunch": 18.042851,
          'Cheerios': 50.764999,
          'Cinnamon Toast Crunch': 19.823573,
          'Clusters': 40.400208,
          'Cocoa Puffs': 22.736446,
          'Corn Chex': 41.445019,
          'Corn Flakes': 45.863324,
          'Corn Pops': 35.782791,
          'Count Chocula': 22.396513,
          "Cracklin' Oat Bran": 40.448772,
          'Cream of Wheat (Quick)': 64.533816,
          'Crispix': 46.895644,
          'Crispy Wheat & Raisins': 36.176196,
          'Double Chex': 44.330856,
          'Froot Loops': 32.207582,
          'Frosted Flakes': 31.435973,
         

In [13]:
def predict(inst,tree):
    #This function is used to predict for any input variable
    
    #Recursively we go through the tree that we built earlier
    
    for nodes in tree.keys():
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;
            
        return prediction
    

In [14]:
inst = df.iloc[6] #this takes row tith index 6

In [15]:
display(inst)

name        Apple Jacks
mfr                   K
type                  C
calories            110
protein               2
fat                   0
sodium              125
fiber               1.0
carbo              11.0
sugars               14
potass               30
vitamins             25
shelf                 2
weight              1.0
cups                1.0
rating        33.174094
Name: 6, dtype: object

In [16]:
#Get prediction
prediction = predict(inst, tree)
prediction

In [17]:
data = {'name' : '100% Bran','mfr' : 'N','calories' : '70','protein' : '4','fat' : '1','sodium' : '130','fiber' :  '10.0','carbo' : '5.0','sugars' : '6','potass' :  '280','vitamins': '25','shelf' : '3','weight' : '1.0','cups' : '0.33'}

In [18]:
inst = pd.Series(data)

In [48]:
#Get prediction
prediction = predict(inst, tree)
display(prediction)

None

In [98]:
# Importing Libraries
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


#Load data and store it into padnas DataFrame objects
X = df.iloc[:, 0:13].values
y = df.iloc[:, 13:14].values

In [99]:
#Separate train and test data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state = 42)

In [100]:
from sklearn.tree import DecisionTreeRegressor

regr=DecisionTreeRegressor(random_state=0)#29
regr.fit(x_train,y_train)

predictions =regr.predict(X_test)

print("Decision Tree Testing score:",regr.score(X_test,y_test))
plt.scatter(y_test,predictions ,color='red')
plt.xlabel("True Values")
plt.ylabel("Predictions")

ValueError: could not convert string to float: 'Bran Flakes'

In [101]:
from sklearn.tree import export_graphviz  
  
# export the decision tree to a tree.dot file 
# for visualizing the plot easily anywhere 

export_graphviz(regr, out_file ='treelimited.dot', 
               feature_names =['mfr','calories','protein','fat','sodium','fiber','carbo','sugars','potass','vitamins','shelf','weight','cups'],
                class_names = 'rating',
                rounded = True, proportion = False, precision = 2, filled = True)

(graph,) = pydot.graph_from_dot_file('treelimited.dot')
Image(graph.create_png())

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.